<a href="https://colab.research.google.com/github/GruffGemini/ComputationalLinguistics/blob/main/%D0%94%D0%97%208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Домашнее задание № 8

## Задание 1

In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from collections import Counter
from matplotlib import pyplot as plt
from string import punctuation
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

punkt = punctuation + '«—»'
ROOT_DIR = '/content/drive/MyDrive'

In [2]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punkt) for token in tokens]
    return [token for token in tokens if token and token not in punkt]

In [3]:
data = pd.read_csv(os.path.join(ROOT_DIR, 'lenta_40k.csv'))
processed_texts = []
vocab = Counter()
for text in data.text:
    processed_text = preprocess(text)
    vocab.update(processed_text)
    processed_texts.append(processed_text)

In [4]:
word2id = {'PAD': 0, 'UNK': 1}
for word in vocab:
    word2id[word] = len(word2id)
id2word = {i: word for word, i in word2id.items()}
X = []
for text in processed_texts:
    ids = [word2id.get(token, 1) for token in text]
    X.append(ids)
MEAN_LEN = np.median([len(x) for x in X])
MAX_LEN = int(MEAN_LEN + 30)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)
id2label = {i: label for i, label in enumerate(set(data.topic.values))}
label2id = {l: i for i, l in id2label.items()}
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

In [5]:
class ModelData:
  def __init__(self, model, name):
    self.model = model
    self.name = name
    self.train_score = 0
    self.val_score = 0

Опишем общие блоки для всех моделей

In [6]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=40)(inputs, )

In [7]:
def make_model(last_layer):
  dense = tf.keras.layers.Dense(64, activation='relu')(last_layer)
  outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])
  return model

In [8]:
def fit_model(model):
  model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
         epochs=15)

Теперь создадим несколько моделей. Для чистоты эксперимента везде пришлось взять число нейронов = 32. Это мало, но при большем числе нейронов не запускается модель с 50 слоями.

In [13]:
models = []

In [14]:
gru = tf.keras.layers.GRU(32, return_sequences=False)(embeddings)
models.append(ModelData(model=make_model(gru), name='1 слой GRU'))

In [15]:
lstm = tf.keras.layers.LSTM(32, return_sequences=False)(embeddings)
models.append(ModelData(model=make_model(lstm), name='1 слой LSTM'))

In [16]:
gru = tf.keras.layers.GRU(32, return_sequences=True)(embeddings)
lstm = tf.keras.layers.LSTM(32, return_sequences=False)(gru)
models.append(ModelData(model=make_model(lstm), name='1 GRU + LSTM'))

In [17]:
bigru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(embeddings)
lstm1 = tf.keras.layers.LSTM(32, return_sequences=True)(bigru)
lstm2 = tf.keras.layers.LSTM(32, return_sequences=False)(lstm1)
models.append(ModelData(model=make_model(lstm2), name='BIGRU + 2 LSTM'))

In [18]:
gru = tf.keras.layers.GRU(32, return_sequences=True)(embeddings)
for _ in range(4):
  gru = tf.keras.layers.GRU(32, return_sequences=True)(gru)
lstm1 = tf.keras.layers.LSTM(32, return_sequences=True)(gru)
lstm2 = tf.keras.layers.LSTM(32, return_sequences=True)(lstm1)
lstm3 = tf.keras.layers.LSTM(32, return_sequences=False)(lstm2)
models.append(ModelData(model=make_model(lstm3), name='5 GRU + 3 LSTM'))

In [19]:
bigru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True),
                                      backward_layer=tf.keras.layers.GRU(32, return_sequences=True,
                                                                         go_backwards=True))(embeddings)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False),
                                      backward_layer=tf.keras.layers.LSTM(32, return_sequences=False,
                                                                         go_backwards=True))(bigru)
models.append(ModelData(model=make_model(bilstm), name='BIGRU + BILSTM'))                                          

In [20]:
lstm1 = tf.keras.layers.LSTM(32, return_sequences=True)(embeddings)
gru1 = tf.keras.layers.GRU(32, return_sequences=True)(lstm1)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(gru1)
bigru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(bilstm)
gru2 = tf.keras.layers.GRU(32, return_sequences=True)(bigru)
lstm2 = tf.keras.layers.LSTM(32, return_sequences=False)(gru2)
models.append(ModelData(model=make_model(lstm2), name='LSTM -> GRU -> BILSTM -> BIGRU -> GRU -> LSTM'))    

In [21]:
from random import choice
layer = choice([tf.keras.layers.SimpleRNN(32, return_sequences=True)(embeddings),
                 tf.keras.layers.LSTM(32, return_sequences=True)(embeddings),
                 tf.keras.layers.GRU(32, return_sequences=True)(embeddings)])
for _ in range(48):
  layer = choice([tf.keras.layers.SimpleRNN(32, return_sequences=True)(layer),
                 tf.keras.layers.LSTM(32, return_sequences=True)(layer),
                 tf.keras.layers.GRU(32, return_sequences=True)(layer)])
layer = choice([tf.keras.layers.SimpleRNN(32, return_sequences=False)(layer),
                 tf.keras.layers.LSTM(32, return_sequences=False)(layer),
                 tf.keras.layers.GRU(32, return_sequences=False)(layer)])
models.append(ModelData(model=make_model(layer), name='50 random layers'))   

Запустим обучение

In [ ]:
for model in models:
  print(model.name)
  fit_model(model.model)

Вывод опущен, так как его слишком много. Оценим итоговую f-меру моделей

In [31]:
for model in models:
  model.val_score = model.model.history.history['val_f1'][-1]
  model.train_score = model.model.history.history['f1'][-1]
sorted_models = sorted(models, key=lambda x: x.val_score, reverse=True)
for model in sorted_models:
  print(f'{model.name} - train={model.train_score} - val={model.val_score}')

1 слой LSTM - train=0.9956343173980713 - val=0.5349631905555725
BIGRU + BILSTM - train=0.9966739416122437 - val=0.5271187424659729
BIGRU + 2 LSTM - train=0.9968664646148682 - val=0.4976016581058502
1 GRU + LSTM - train=0.9963577389717102 - val=0.491421103477478
1 слой GRU - train=0.9897481203079224 - val=0.48922500014305115
LSTM -> GRU -> BILSTM -> BIGRU -> GRU -> LSTM - train=0.9902581572532654 - val=0.47209644317626953
5 GRU + 3 LSTM - train=0.9398456811904907 - val=0.44356808066368103
50 random layers - train=0.0 - val=0.0


Из результатов видно, что лучшая модель - самая простая с 1 слоем LSTM. Нагромождение слоёв не даёт прироста результата, а модель с 50 слоями вообще не работает адекватно (она вроде бы обучалась, но очень медленно, 15 эпох ей явно было мало).

## Задание 2

In [ ]:
!pip install datasets
!pip install navec
!pip install slovnet

In [123]:
from datasets import load_dataset
from navec import Navec
from slovnet.model.emb import NavecEmbedding

In [ ]:
dataset = load_dataset("wikiann", 'ru')

In [146]:
vocab = Counter()
for sent in dataset['train']['tokens']:
  vocab.update([x.lower() for x in sent])
word2id = {'<pad>':0, '<unk>':1}
for word in vocab:
    word2id[word] = len(word2id)
id2word = {i:word for word, i in word2id.items()}
X = []
for sent in dataset['train']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)
X_test = []
for sent in dataset['test']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X_test.append(ids)
MAX_LEN = max(len(x) for x in X)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_LEN, padding='post')
id2labels = {0:'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5: 'B-LOC', 6:'I-LOC', 7:'<pad>'}
label2id = {v:k for k,v in id2labels.items()} 
y = tf.keras.preprocessing.sequence.pad_sequences(dataset['train']['ner_tags'], value=7,
                                                  maxlen=MAX_LEN,  padding='post')
y_test = tf.keras.preprocessing.sequence.pad_sequences(dataset['test']['ner_tags'], value=7,
                                                       maxlen=MAX_LEN,  padding='post')

Используем предобученные эмбеддинги navec

In [147]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(os.path.join(ROOT_DIR, path))
emb = NavecEmbedding(navec)
num_tokens = len(vocab) + 2
embedding_dim = 300

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for id in id2word:
  try:
    navec_id = navec.vocab[id2word[id]]
    input = np.asarray(navec_id)    
    embedding_vector = emb(input)
  except KeyError:
    embedding_vector = navec.vocab['<unk>']
  embedding_matrix[id] = embedding_vector

Создадим Embedding слой на основе предобученных эмбеддингов

In [148]:
embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Обучим BILSTM модель

In [171]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = embedding_layer(inputs)

bilstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embeddings)
bilstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(bilstm_1)
bilstm_3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(bilstm_2)
bilstm_4 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(bilstm_3)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(bilstm_4)

bilstm_model = tf.keras.Model(inputs=inputs, outputs=outputs)
bilstm_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

In [172]:
bilstm_model.fit(X, y, 
                 validation_data=(X_test, y_test),
                 batch_size=64,
                 epochs=5)

Epoch 1/5
313/313 [==============================] - 26s 52ms/step - loss: 0.1450 - accuracy: 0.9578 - val_loss: 0.0862 - val_accuracy: 0.9716
Epoch 2/5
313/313 [==============================] - 16s 50ms/step - loss: 0.0602 - accuracy: 0.9812 - val_loss: 0.0658 - val_accuracy: 0.9782
Epoch 3/5
313/313 [==============================] - 14s 43ms/step - loss: 0.0465 - accuracy: 0.9854 - val_loss: 0.0592 - val_accuracy: 0.9810
Epoch 4/5
313/313 [==============================] - 13s 43ms/step - loss: 0.0395 - accuracy: 0.9876 - val_loss: 0.0604 - val_accuracy: 0.9807
Epoch 5/5
313/313 [==============================] - 14s 46ms/step - loss: 0.0347 - accuracy: 0.9891 - val_loss: 0.0590 - val_accuracy: 0.9826


И BIGRU модель

In [173]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = embedding_layer(inputs)

bigru_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embeddings)
bigru_2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(bigru_1)
bigru_3 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(bigru_2)
bigru_4 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(bigru_3)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(bigru_4)

bigru_model = tf.keras.Model(inputs=inputs, outputs=outputs)
bigru_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

In [174]:
bigru_model.fit(X, y, 
                 validation_data=(X_test, y_test),
                 batch_size=64,
                 epochs=5)

Epoch 1/5
313/313 [==============================] - 26s 50ms/step - loss: 0.1193 - accuracy: 0.9677 - val_loss: 0.0711 - val_accuracy: 0.9763
Epoch 2/5
313/313 [==============================] - 13s 41ms/step - loss: 0.0521 - accuracy: 0.9835 - val_loss: 0.0609 - val_accuracy: 0.9794
Epoch 3/5
313/313 [==============================] - 14s 43ms/step - loss: 0.0426 - accuracy: 0.9865 - val_loss: 0.0581 - val_accuracy: 0.9811
Epoch 4/5
313/313 [==============================] - 13s 40ms/step - loss: 0.0374 - accuracy: 0.9881 - val_loss: 0.0547 - val_accuracy: 0.9821
Epoch 5/5
313/313 [==============================] - 13s 40ms/step - loss: 0.0318 - accuracy: 0.9899 - val_loss: 0.0535 - val_accuracy: 0.9829


Метрики для BILSTM

In [177]:
from sklearn.metrics import classification_report
pred = bilstm_model.predict(X_test).argmax(2)
print(classification_report(y_test.reshape(-1), pred.reshape(-1), labels=list(id2labels.keys()),
                                                                     target_names=list(id2labels.values()),
                                                                     zero_division=0))

              precision    recall  f1-score   support

           O       0.91      0.95      0.93     40480
       B-PER       0.75      0.90      0.82      3542
       I-PER       0.82      0.95      0.88      7544
       B-ORG       0.65      0.65      0.65      4074
       I-ORG       0.88      0.67      0.76      8008
       B-LOC       0.82      0.71      0.76      4560
       I-LOC       0.90      0.62      0.73      3060
       <pad>       1.00      1.00      1.00    468732

    accuracy                           0.98    540000
   macro avg       0.84      0.81      0.82    540000
weighted avg       0.98      0.98      0.98    540000



Метрики для BIGRU

In [178]:
from sklearn.metrics import classification_report
pred = bigru_model.predict(X_test).argmax(2)
print(classification_report(y_test.reshape(-1), pred.reshape(-1), labels=list(id2labels.keys()),
                                                                     target_names=list(id2labels.values()),
                                                                     zero_division=0))

              precision    recall  f1-score   support

           O       0.94      0.92      0.93     40480
       B-PER       0.79      0.90      0.84      3542
       I-PER       0.83      0.95      0.89      7544
       B-ORG       0.63      0.68      0.65      4074
       I-ORG       0.84      0.73      0.78      8008
       B-LOC       0.76      0.80      0.78      4560
       I-LOC       0.75      0.76      0.75      3060
       <pad>       1.00      1.00      1.00    468732

    accuracy                           0.98    540000
   macro avg       0.82      0.84      0.83    540000
weighted avg       0.98      0.98      0.98    540000



По метрикам BIGRU немного лучше

In [179]:
import re

def tokenize(text, word2id):
    # токенизирует и переводит в индексы
    tokens = re.findall('\w+|[^\w\s]+', text)
    ids = [word2id.get(token.lower(), 1) for token in tokens]
    return tokens, ids

def pred2tags(pred, id2label, length):
    # декодирует индексы в части речи
    # length нужно чтобы откидывать паддинги или некорректные предсказания
    pred = pred.argmax(2)[0, :length]
    labels = [id2label[l] for l in pred]
    return labels

def label_seq(text, word2id, id2label, max_len, model):
    tokens, ids = tokenize(text, word2id)
    pred = model.predict(tf.keras.preprocessing.sequence.pad_sequences([ids], 
                                                                       maxlen=max_len, 
                                                                       padding='post'))
    labels = pred2tags(pred, id2label, len(ids))
    
    return list(zip(tokens, labels))

Попробуем сравнить модели на предложениях

In [185]:
def compare(sentence):
  print('BILSTM')
  print(label_seq(sentence, word2id, id2labels, MAX_LEN, bilstm_model))
  print('BIGRU')
  print(label_seq(sentence, word2id, id2labels, MAX_LEN, bigru_model))

In [186]:
compare('Алексей сказал Светлане, чтобы она собиралась на поезд в Москву.')

BILSTM
[('Алексей', 'B-PER'), ('сказал', 'I-PER'), ('Светлане', 'I-PER'), (',', 'O'), ('чтобы', 'O'), ('она', 'O'), ('собиралась', 'O'), ('на', 'O'), ('поезд', 'O'), ('в', 'O'), ('Москву', 'B-LOC'), ('.', 'O')]
BIGRU
[('Алексей', 'B-PER'), ('сказал', 'I-PER'), ('Светлане', 'I-PER'), (',', 'O'), ('чтобы', 'O'), ('она', 'O'), ('собиралась', 'O'), ('на', 'O'), ('поезд', 'O'), ('в', 'O'), ('Москву', 'B-LOC'), ('.', 'O')]


Идентичные результаты

In [187]:
compare('Я поступил в ВШЭ в Питере')

BILSTM
[('Я', 'O'), ('поступил', 'O'), ('в', 'O'), ('ВШЭ', 'B-ORG'), ('в', 'O'), ('Питере', 'B-ORG')]
BIGRU
[('Я', 'O'), ('поступил', 'O'), ('в', 'O'), ('ВШЭ', 'B-ORG'), ('в', 'I-ORG'), ('Питере', 'B-LOC')]


У обеих моделях по одной ошибке в разных местах

In [188]:
compare('Виктор, Семен, Петр, Арагорн')

BILSTM
[('Виктор', 'B-PER'), (',', 'I-PER'), ('Семен', 'I-PER'), (',', 'O'), ('Петр', 'I-PER'), (',', 'I-PER'), ('Арагорн', 'I-PER')]
BIGRU
[('Виктор', 'B-PER'), (',', 'I-PER'), ('Семен', 'I-PER'), (',', 'I-PER'), ('Петр', 'I-PER'), (',', 'I-PER'), ('Арагорн', 'I-PER')]


BILSTM посчитал "Виктор, Семен" за одну сущность, "Петр, Арагорн" - за другую

BIGRU объединил в одну сущность все имена

In [194]:
compare('Президент Международного олимпийского комитета Томас Бах объявил зимние Олимпийские игры 2022 года закрытыми.')

BILSTM
[('Президент', 'O'), ('Международного', 'B-ORG'), ('олимпийского', 'I-ORG'), ('комитета', 'O'), ('Томас', 'B-PER'), ('Бах', 'I-PER'), ('объявил', 'O'), ('зимние', 'B-ORG'), ('Олимпийские', 'I-ORG'), ('игры', 'O'), ('2022', 'B-ORG'), ('года', 'I-ORG'), ('закрытыми', 'I-ORG'), ('.', 'O')]
BIGRU
[('Президент', 'O'), ('Международного', 'B-ORG'), ('олимпийского', 'I-ORG'), ('комитета', 'I-ORG'), ('Томас', 'B-PER'), ('Бах', 'I-PER'), ('объявил', 'O'), ('зимние', 'B-ORG'), ('Олимпийские', 'I-ORG'), ('игры', 'O'), ('2022', 'B-ORG'), ('года', 'I-ORG'), ('закрытыми', 'I-ORG'), ('.', 'O')]


Здесь BIGRU отработала удачнее, корректно определив организацию "Международного олимпийского комитета". Однако обе модели ошибочно отнесли к организациям фразы в конце предложения.

In [195]:
compare('Бывший нападающий «Динамо» Сильвестр Игбун подписал контракт с «Нижним Новгородом». Об этом «Спорт-Экспрессу» сообщил генеральный директор клуба Равиль Измайлов.')

BILSTM
[('Бывший', 'O'), ('нападающий', 'O'), ('«', 'O'), ('Динамо', 'B-ORG'), ('»', 'O'), ('Сильвестр', 'B-PER'), ('Игбун', 'I-PER'), ('подписал', 'O'), ('контракт', 'O'), ('с', 'O'), ('«', 'O'), ('Нижним', 'B-PER'), ('Новгородом', 'I-PER'), ('».', 'I-PER'), ('Об', 'O'), ('этом', 'O'), ('«', 'O'), ('Спорт', 'O'), ('-', 'O'), ('Экспрессу', 'O'), ('»', 'O'), ('сообщил', 'O'), ('генеральный', 'O'), ('директор', 'O'), ('клуба', 'O'), ('Равиль', 'B-PER'), ('Измайлов', 'I-PER'), ('.', 'O')]
BIGRU
[('Бывший', 'O'), ('нападающий', 'O'), ('«', 'O'), ('Динамо', 'B-ORG'), ('»', 'O'), ('Сильвестр', 'B-PER'), ('Игбун', 'I-PER'), ('подписал', 'O'), ('контракт', 'O'), ('с', 'O'), ('«', 'O'), ('Нижним', 'B-PER'), ('Новгородом', 'I-PER'), ('».', 'I-PER'), ('Об', 'O'), ('этом', 'O'), ('«', 'O'), ('Спорт', 'B-ORG'), ('-', 'O'), ('Экспрессу', 'B-ORG'), ('»', 'O'), ('сообщил', 'O'), ('генеральный', 'O'), ('директор', 'O'), ('клуба', 'O'), ('Равиль', 'B-ORG'), ('Измайлов', 'I-ORG'), ('.', 'O')]


Обе модели ошибочно определили "Нижний Новгород" в I-PER. BIGRU верно выделила "Спорт-Экспресс" как организацию, но в отличие от BILSTM ошиблась, посчитав "Равиля Измайлова" организацией.

In [197]:
compare('Виктор Наворски прилетает в Нью-Йорк, но во время полёта в его родной стране Кракожии (вымышленная славяноязычная страна Восточной Европы) произошёл военный переворот и страна прекратила своё существование.')

BILSTM
[('Виктор', 'B-PER'), ('Наворски', 'I-PER'), ('прилетает', 'O'), ('в', 'O'), ('Нью', 'B-LOC'), ('-', 'O'), ('Йорк', 'B-LOC'), (',', 'O'), ('но', 'O'), ('во', 'O'), ('время', 'O'), ('полёта', 'O'), ('в', 'O'), ('его', 'O'), ('родной', 'O'), ('стране', 'O'), ('Кракожии', 'O'), ('(', 'O'), ('вымышленная', 'O'), ('славяноязычная', 'O'), ('страна', 'B-LOC'), ('Восточной', 'I-LOC'), ('Европы', 'I-LOC'), (')', 'O'), ('произошёл', 'O'), ('военный', 'I-ORG'), ('переворот', 'I-ORG'), ('и', 'O'), ('страна', 'O'), ('прекратила', 'O'), ('своё', 'O'), ('существование', 'O'), ('.', 'O')]
BIGRU
[('Виктор', 'B-PER'), ('Наворски', 'I-PER'), ('прилетает', 'O'), ('в', 'O'), ('Нью', 'B-LOC'), ('-', 'O'), ('Йорк', 'B-LOC'), (',', 'O'), ('но', 'O'), ('во', 'O'), ('время', 'O'), ('полёта', 'O'), ('в', 'O'), ('его', 'O'), ('родной', 'O'), ('стране', 'O'), ('Кракожии', 'O'), ('(', 'O'), ('вымышленная', 'B-LOC'), ('славяноязычная', 'I-LOC'), ('страна', 'I-LOC'), ('Восточной', 'I-LOC'), ('Европы', 'I-LOC')

Обе модели причислили много лишнего к именованным сущностям, BILSTM сделала меньше ошибок